In [2]:
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
import pickle
import matplotlib.pyplot as plt
import cv2
from scipy.interpolate import interp1d
from scipy.spatial.transform import Rotation as R
import ast
import shutil

In [4]:
drive_folder='/media/Blue2TB1/MMSys18'
VIDEO_FOLDER=os.path.join(drive_folder,'Videos')
VIDEO_DATA_FOLDER=os.path.join(drive_folder,'video_data')
OUTPUT_TRUE_SALIENCY_FOLDER = os.path.join(drive_folder,'true_saliency')
VIDEO_IMG_PATH=os.path.join(drive_folder,'5fps_Video_Images')

In [8]:
vid_lens={}
videos=os.listdir(VIDEO_IMG_PATH)
for video in videos:
    tstamps=np.load(os.path.join(VIDEO_IMG_PATH,video,'timestamps.npy'))
    vid_lens[video]=len(tstamps)

In [22]:
for video in videos:
    uvs=np.load(os.path.join(VIDEO_DATA_FOLDER,video,f'{video}_unit_vectors.npy'))
    tstamps=np.load(os.path.join(VIDEO_DATA_FOLDER,video,f'{video}_timestamps.npy'))
    SI_path=os.path.join(VIDEO_DATA_FOLDER,video,f'{video}_SI.npy')
    TI_path=os.path.join(VIDEO_DATA_FOLDER,video,f'{video}_TI.npy')
    TIs=np.load(TI_path)
    SIs=np.load(SI_path)
    #np.save(SI_path,SIs[:100])
    #np.save(TI_path,TIs[:100])

(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)


### Capturing video frames

In [179]:
def store_frames(video_path,output_folder,fps=None,rescale=2):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    #print(f"Saving video {os.path.basename(video_path).split('.')[0]}")
    video=cv2.VideoCapture(os.path.join(video_path))
    original_frame_rate=round(video.get(cv2.CAP_PROP_FPS))
    print(original_frame_rate)
    is_div= original_frame_rate%fps==0
    #vid_num=os.path.basename(video_path).split('.')[0]
    #trajectory_folder=os.path.join("D:/CVPR18/sampled_dataset",vid_num)
    #files = os.listdir(trajectory_folder)
    #first_file = os.path.join(trajectory_folder, files[0])
    #data = pd.read_csv(first_file, header=None) 
    #print(len(data))
    
    #print(is_div)
    original_frame_width=int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_frame_height=int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_duration=1/original_frame_rate
    #desired_timestamps = np.arange(0, int(video.get(cv2.CAP_PROP_FRAME_COUNT)) /original_frame_rate, 0.2)
    #print(desired_timestamps[-1]
    if fps:
        frame_interval=original_frame_rate//fps
    else:
        frame_interval=1
    frame_count=0
    last_frame=int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    #print(frame_duration*last_frame)
    timestamps=[]
    frames=[]
    while video.isOpened():
        ret,frame=video.read()
        if not ret:
            break
        if is_div:
            if frame_count%frame_interval==0 or frame_count+1==last_frame:
                frame=cv2.resize(frame,(original_frame_width//rescale,original_frame_height//rescale))
                frames.append(frame)
                timestamps.append(frame_count*frame_duration)
            #print((frame_count)*frame_duration)
        else:
            frame=cv2.resize(frame,(original_frame_width//rescale,original_frame_height//rescale))
            frames.append(frame)
            timestamps.append(frame_count*frame_duration)
        frame_count+=1
    #print(frame_count)
    if not is_div:
        sampled_frames=[]
        sampled_timestamps=[]
        desired_timestamps=list(np.arange(0,round(timestamps[-1])+0.1,1/fps))
        for ts in desired_timestamps:
            closest_idx=np.abs(timestamps-ts).argmin()
            sampled_frames.append(frames[closest_idx])
            sampled_timestamps.append(timestamps[closest_idx])
        timestamps=sampled_timestamps
        frames=sampled_frames
    #print(len(timestamps))
    #if len(data)!=len(timestamps):
    #    print(vid_num)
    #print(desired_timestamps)
    video.release()
    #for i,frame in enumerate(frames):
        #np.save(f'{output_folder}/{i}.npy',frame)
    np.save(f'{output_folder}/sampled_video.npy',frames)
    np.save(f'{output_folder}/timestamps.npy',timestamps)
    
    #print(f"Saved video {video_path[-6:-3]}")

In [180]:
dataset_folder="D:/Jin_22"
video_folder=os.path.join(dataset_folder,'Videos')
videos=[d.split('.')[0] for d in os.listdir(video_folder) if os.path.isfile(os.path.join(video_folder,d))]
#videos=sorted(videos,key=lambda x:int(x.split('_')[1]))
sample_videos=False
output_dir=video_folder
ct=0
output_dir=os.path.join(dataset_folder,"5fps_Video_Images")
if sample_videos:    
    for video in videos:
        video_path=os.path.join(video_folder,video+".mp4")
        #video_path=os.path.join(video_folder,"video_1.mp4")
        output_folder=os.path.join(output_dir,video)
        store_frames(video_path,output_folder, fps=5)
    

### Resampling User Trajectories

In [181]:
users_folder=os.path.join(dataset_folder,"Version2")
video_img_folder=os.path.join(dataset_folder,'5fps_Video_Images')
user_folder=os.path.join(users_folder,'V2 (1)')
trajec_dir=os.path.join(dataset_folder,'Full_Dataset')

In [182]:
def rearrange_folders(base_dir,dest_dir):
    user_folders=sorted(os.listdir(base_dir),key=lambda x:int(x.split(' ')[1][1:-1]))
    for folder in user_folders:
        user_num=folder.split(' ')[1][1:-1]
        user_folder_path=os.path.join(base_dir,folder)
        trajectory_files=os.listdir(os.path.join(user_folder_path))
        for trajectory_file in trajectory_files:
            vid_num=trajectory_file.split('_')[2]
            new_name=f'user-{user_num}_vid-{vid_num}_trajectory.csv'
            vid_folder_path=os.path.join(dest_dir,f'video_{vid_num}')
            if not os.path.exists(vid_folder_path):
                os.makedirs(vid_folder_path)
            
            src_path=os.path.join(user_folder_path,trajectory_file)
            dst_path=os.path.join(vid_folder_path,new_name)
            shutil.copy2(src_path,dst_path)

In [183]:
#rearrange_folders(users_folder,trajec_dir)

In [184]:
def interpolate_position(timestamps,positions,new_timestamps):
    interp_fun=interp1d(timestamps,positions,axis=0,kind='linear',fill_value='extrapolate')
    return interp_fun(new_timestamps)

def interpolate_quat(timestamps,quaternions,new_timestamps):
    slerp=R.from_quat(quaternions)
    key_rots=slerp.as_quat()
    interp_fun=interp1d(timestamps,key_rots,axis=0,kind='linear', fill_value='extrapolate')
    interpolated_rots=interp_fun(new_timestamps)
    return R.from_quat(interpolated_rots).as_quat()

def interpolate_unit_vector(timestamps, vectors, new_timestamps):
    interp_func = interp1d(timestamps, vectors, axis=0, kind='linear', fill_value='extrapolate')
    interpolated_vectors = interp_func(new_timestamps)
    norm = np.linalg.norm(interpolated_vectors, axis=1, keepdims=True)
    normalized_vectors = interpolated_vectors / norm
    return normalized_vectors

In [185]:
def load_timestamps(video_folder):
    video_timestamps={}
    for root,dirs,files in os.walk(video_folder):
        for dir in dirs:
            video_number=int(dir.split('_')[1])
            timestamps_path=os.path.join(root,dir,'timestamps.npy')
            if os.path.exists(timestamps_path):
                video_timestamps[video_number]=np.load(timestamps_path)
    return video_timestamps

def get_video_number(file_name):
    vid_number=int(file_name.split('_')[2])
    return vid_number

In [186]:
def process_csv(file_path, new_timestamps):
    #print(file_path)
    data=pd.read_csv(file_path)
    time=data['AdjustedTime']
    #print(time[0])
    time=time-time[0]
    #print(time[0])
    pose_position=np.array(data['Pose_Position'].apply(ast.literal_eval).tolist())
    pose_rotation=np.array(data['Pose_Rotation'].apply(ast.literal_eval).tolist())
    unit_vector=np.array(data['Unit_Vector'].apply(ast.literal_eval).tolist())
    #print(pose_position)
    #print(pose_rotation[23,:],pose_rotation[24,:],pose_rotation[25,:])
    #print(len(time))
    #print(len(pose_rotation))
    interpolated_position=interpolate_position(time,pose_position,new_timestamps)
    interpolated_rotation=interpolate_quat(time,pose_rotation,new_timestamps)
    interpolated_unit_vector=interpolate_unit_vector(time,unit_vector,new_timestamps)
    #print(interpolated_position.tolist())
    #print(interpolated_position[1,:])
    """ sampled_data = pd.DataFrame({
        'Timestamp': new_timestamps,
        'Pose_Position': interpolated_position.tolist(),
        'Pose_Rotation': interpolated_rotation.tolist(),
        'Unit_Vector': interpolated_unit_vector.tolist(),
    }) """
    
    return interpolated_position,interpolated_rotation,interpolated_unit_vector

In [187]:
def save_numpy_arrays(video_output_dir, vid_name, user_data):
    timestamps, users, positions, rotations, vectors = user_data
    np.save(os.path.join(video_output_dir, f"{vid_name}_users.npy"), users)
    np.save(os.path.join(video_output_dir, f"{vid_name}_timestamps.npy"), timestamps)
    np.save(os.path.join(video_output_dir, f"{vid_name}_positions.npy"), positions)
    np.save(os.path.join(video_output_dir, f"{vid_name}_rotations.npy"), rotations)
    np.save(os.path.join(video_output_dir, f"{vid_name}_unit_vectors.npy"), vectors)

In [188]:
dataset_folder='D:/Jin_22'
trajec_dir=os.path.join(dataset_folder,'Full_Dataset')
users_folder=os.path.join(dataset_folder,"Version2")
video_img_folder=os.path.join(dataset_folder,'5fps_Video_Images')
video_path=os.path.join(dataset_folder,'Videos')
agg_folder=os.path.join(dataset_folder,'Trajectory_info')
sampled_trajectories=os.path.join(dataset_folder,'sampled_dataset')

In [189]:
tstamps=load_timestamps(video_img_folder)
output_dir="D:/Jin_22/Trajectory_info"
videos=os.listdir(trajec_dir)
def resample_trajectories(src_dir,dst_dir,video_img_folder):
    tstamps=load_timestamps(video_img_folder)
    videos=os.listdir(src_dir)
    for vid in videos:
        vid_path=os.path.join(src_dir,vid)
        vid_num=vid.split("_")[1]
        timestamps=tstamps[int(vid_num)]
        output_dir=os.path.join(dst_dir,vid)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        csv_files=[f for f in os.listdir(vid_path) if f.endswith('.csv')]
        for f in csv_files:
            traj_path=os.path.join(vid_path,f)
            pos,rots,unit_vecs=process_csv(traj_path,timestamps)
            sampled_data=pd.DataFrame({
                'Timestamp':timestamps,
                'Pose_Position': pos.tolist(),
                'Pose_Rotation': rots.tolist(),
                'Unit_Vector':unit_vecs.tolist()
            })
            dst_path=os.path.join(dst_dir,vid)
            sampled_data.to_csv(dst_path,index=False)

In [190]:
#resample_trajectories(trajec_dir,sampled_trajectories,video_img_folder)

In [191]:
def get_user_num(filename):
    num=filename.split('_')[0].split('-')[1]
    return int(num)

def get_user_list(traj_names):
    user_names=[]
    for traj_name in traj_names:
        user_name=traj_name.split('_')[0]
        user_names.append(user_name)
    return user_names

def store_trajectory_aggregates(src_dir,dst_dir):
    videos=os.listdir(src_dir)
    print(len(videos))
    for video in videos:
        trajs=os.listdir(os.path.join(src_dir,video))
        print(f'Processing {video}')
        op_dir=os.path.join(dst_dir,video)
        if not os.path.exists(op_dir):
            os.makedirs(op_dir)
        poses=[]
        rots=[]
        uvs=[]
        trajs=sorted(trajs,key=get_user_num)
        users=get_user_list(trajs)
        for traj in trajs:
            path=os.path.join(src_dir,video,traj)
            df=pd.read_csv(path)
            tstamps=np.array(df['Timestamp'])
            pos=np.array(df['Pose_Position'].apply(ast.literal_eval).tolist())
            rot=np.array(df['Pose_Rotation'].apply(ast.literal_eval).tolist())
            uv=np.array(df['Unit_Vector'].apply(ast.literal_eval).tolist())
            poses.append(pos)
            rots.append(rot)
            uvs.append(uv)
        uvs=np.array(uvs)
        poses=np.array(poses)
        rots=np.array(rots)

        save_numpy_arrays(op_dir,video,(tstamps,users,poses,rots,uvs))

In [192]:
store_trajectory_aggregates(sampled_trajectories,agg_folder)

27
Processing video_10
Processing video_11
Processing video_12
Processing video_13
Processing video_14
Processing video_15
Processing video_16
Processing video_17
Processing video_18
Processing video_19
Processing video_1
Processing video_20
Processing video_21
Processing video_22
Processing video_23
Processing video_24
Processing video_25
Processing video_26
Processing video_27
Processing video_2
Processing video_3
Processing video_4
Processing video_5
Processing video_6
Processing video_7
Processing video_8
Processing video_9


In [ ]:
test2=np.load('D:/Jin_22/Trajectory_info/Video_7/Video_7_rotations.npy')
test=np.load('D:/Jin_22/Trajectory_info/Video_10/Video_10_rotations.npy')

### CVPR18

In [ ]:
import csv

In [235]:
dataset_folder='D:/CVPR18'
sampled_dataset=os.path.join(dataset_folder,'sampled_dataset')
trajec_dir=os.path.join(dataset_folder,'Trajectory_info')
video_folder=os.path.join(dataset_folder,'Videos')

def get_user_num(filename):
    user_num=int(filename[1:])
    return user_num

def store_numpy(src_dir,dst_dir):
    videos=os.listdir(src_dir)
    for video in videos:
        vid_path=os.path.join(src_dir,video)
        users=os.listdir(vid_path)
        dst_path=os.path.join(dst_dir,video)
        #print(video)
        if not os.path.exists(dst_path):
            os.makedirs(dst_path)
        uvs=[]
        tstamps=[]
        users=sorted(users,key=get_user_num)
        for user in users:
            traj_path=os.path.join(vid_path,user)
            traj=np.loadtxt(traj_path,delimiter=',')
            #print(traj.shape)
            tstamps=traj[:,0]
            uv=traj[:,1:]
            #print(uv.shape)
            uvs.append(uv)
        shortest=len(min(uvs,key=len))
        longest=len(max(uvs,key=len))
        if shortest!=longest:
            print(f'{video}: {longest-shortest} : {longest}')
            prob=np.array([True,shortest,longest])
            np.save(os.path.join(dst_path,f'{video}_len_correction.npy'),prob)
        uvs=np.array([uv[:shortest] for uv in uvs])
        np.save(os.path.join(dst_path,f'{video}_timestamps.npy'),tstamps)
        np.save(os.path.join(dst_path,f'{video}_unit_vectors.npy'),uvs)
        np.save(os.path.join(dst_path,f'{video}_users.npy'),np.array(users))

In [236]:
store_numpy(sampled_dataset,trajec_dir)

006: 1 : 293
059: 1 : 163
066: 1 : 155
070: 6 : 108
074: 1 : 238
087: 1 : 125
090: 1 : 108
094: 18 : 108
098: 1 : 105
102: 1 : 124
118: 1 : 196
119: 5 : 173
120: 12 : 290
121: 5 : 196
123: 1 : 169
124: 2 : 77
125: 2 : 196
127: 1 : 237
128: 3 : 295
129: 1 : 234
131: 1 : 203
132: 33 : 259
133: 49 : 196
134: 42 : 167
135: 16 : 161
137: 5 : 134
138: 1 : 183
139: 1 : 193
140: 7 : 141
142: 2 : 199
153: 9 : 243
159: 1 : 158
166: 8 : 108
167: 1 : 144
169: 3 : 194
171: 13 : 204
172: 4 : 256
174: 3 : 219
175: 2 : 229
176: 27 : 256
177: 24 : 169
187: 1 : 203
189: 3 : 182
198: 1 : 217
203: 1 : 247
204: 1 : 221
211: 1 : 146
212: 1 : 267
213: 2 : 151
214: 2 : 152
215: 3 : 173


### NOSSDAV_17

In [7]:
dataset_folder='D:/Fan_NOSSDAV_17'
sampled_dataset=os.path.join(dataset_folder,'sampled_dataset')
trajec_dir=os.path.join(dataset_folder,'Trajectory_info')
video_folder=os.path.join(dataset_folder,'Videos')

def get_user_num(filename):
    user_num=int(filename[-2:])
    return user_num

def store_numpy(src_dir,dst_dir):
    videos=os.listdir(src_dir)
    for video in videos:
        vid_path=os.path.join(src_dir,video)
        users=os.listdir(vid_path)
        dst_path=os.path.join(dst_dir,video)
        #print(video)
        if not os.path.exists(dst_path):
            os.makedirs(dst_path)
        uvs=[]
        tstamps=[]
        users=sorted(users,key=get_user_num)
        for user in users:
            traj_path=os.path.join(vid_path,user)
            traj=np.loadtxt(traj_path,delimiter=',')
            #print(traj.shape)
            tstamps=traj[:,0]
            uv=traj[:,1:]
            #print(uv.shape)
            uvs.append(uv)
        
        shortest=len(min(uvs,key=len))
        longest=len(max(uvs,key=len))
        if shortest!=longest:
            print(f'{video}: {longest-shortest} : {longest}')
            prob=np.array([True,shortest,longest])
            np.save(os.path.join(dst_path,f'{video}_len_correction.npy'),prob)
        uvs=np.array([uv[:shortest] for uv in uvs])
        np.save(os.path.join(dst_path,f'{video}_timestamps.npy'),tstamps)
        np.save(os.path.join(dst_path,f'{video}_unit_vectors.npy'),uvs)
        np.save(os.path.join(dst_path,f'{video}_users.npy'),np.array(users))

In [8]:
store_numpy(sampled_dataset,trajec_dir)

### PAMI

In [29]:
dataset_folder='D:/PAMI18'
sampled_dataset=os.path.join(dataset_folder,'sampled_dataset')
trajec_dir=os.path.join(dataset_folder,'video_data')
video_folder=os.path.join(dataset_folder,'Videos')

def get_user_num(filename):
    user_num=int(filename[-2:])
    return user_num

def store_numpy(src_dir,dst_dir):
    videos=os.listdir(src_dir)
    for video in videos:
        
        vid_path=os.path.join(src_dir,video)
        users=os.listdir(vid_path)
        dst_path=os.path.join(dst_dir,video)
        #print(video)
        if not os.path.exists(dst_path):
            os.makedirs(dst_path)
        uvs=[]
        tstamps=[]
        users=sorted(users)
        for user in users:
            traj_path=os.path.join(vid_path,user)
            traj=np.loadtxt(traj_path,delimiter=',')
            #print(traj.shape)
            tstamps=traj[:,0]
            uv=traj[:,1:]
            #print(uv.shape)
            uvs.append(uv)
        
        shortest=len(min(uvs,key=len))
        longest=len(max(uvs,key=len))
        if shortest!=longest:
            print(f'{video}: {longest-shortest} : {longest}')
            prob=np.array([True,shortest,longest])
            np.save(os.path.join(dst_path,f'{video}_len_correction.npy'),prob)
        uvs=np.array([uv[:shortest] for uv in uvs])
        np.save(os.path.join(dst_path,f'{video}_timestamps.npy'),tstamps)
        np.save(os.path.join(dst_path,f'{video}_unit_vectors.npy'),uvs)
        np.save(os.path.join(dst_path,f'{video}_users.npy'),np.array(users))

In [30]:
store_numpy(sampled_dataset,trajec_dir)

Waterfall: 1 : 102


### MMSys18

In [38]:
dataset_folder='D:/MMSys18'
sampled_dataset=os.path.join(dataset_folder,'sampled_dataset')
trajec_dir=os.path.join(dataset_folder,'video_data')
video_folder=os.path.join(dataset_folder,'Videos')

def get_user_num(filename):
    user_num=int(filename)
    return user_num

def store_numpy(src_dir,dst_dir):
    videos=os.listdir(src_dir)
    videos=[vid for vid in videos if os.path.isdir(os.path.join(src_dir,vid))]
    for video in videos:
        
        vid_path=os.path.join(src_dir,video)
        users=os.listdir(vid_path)
        dst_path=os.path.join(dst_dir,video)
        print(f'Processing {video}')
        if not os.path.exists(dst_path):
            os.makedirs(dst_path)
        uvs=[]
        tstamps=[]
        users=sorted(users,key=get_user_num)
        for user in users:
            traj_path=os.path.join(vid_path,user)
            traj=np.loadtxt(traj_path,delimiter=',')
            #print(traj.shape)
            tstamps=traj[:,0]
            uv=traj[:,1:]
            #print(uv.shape)
            uvs.append(uv)
        
        shortest=len(min(uvs,key=len))
        longest=len(max(uvs,key=len))
        if shortest!=longest:
            print(f'{video}: {longest-shortest} : {longest}')
            prob=np.array([True,shortest,longest])
            np.save(os.path.join(dst_path,f'{video}_len_correction.npy'),prob)
        uvs=np.array([uv[:shortest] for uv in uvs])
        np.save(os.path.join(dst_path,f'{video}_timestamps.npy'),tstamps)
        np.save(os.path.join(dst_path,f'{video}_unit_vectors.npy'),uvs)
        np.save(os.path.join(dst_path,f'{video}_users.npy'),np.array(users))

In [39]:
store_numpy(sampled_dataset,trajec_dir)

Processing 10_Cows
Processing 11_Abbottsford
Processing 12_TeatroRegioTorino
Processing 13_Fountain
Processing 14_Warship
Processing 15_Cockpit
Processing 16_Turtle
Processing 17_UnderwaterPark
Processing 18_Bar
Processing 19_Touvet
Processing 1_PortoRiverside
Processing 2_Diner
Processing 3_PlanEnergyBioLab
Processing 4_Ocean
Processing 5_Waterpark
Processing 6_DroneFlight
Processing 7_GazaFishermen
Processing 8_Sofa
Processing 9_MattSwift
